# Site-specific Authentication and Federated Job-level Authorization

Site-specific authentication and authorization allows users to inject their own authentication and authorization methods into the NVFlare system. This includes the FL server / clients registration, authentication, and the job deployment and run authorization.

NVFlare provides a general purpose event based pluggable authentication and authorization framework to allow for expanding functionality such as:

* exposing the app through a WAF (Web Application Firewall) or any other network element enforcing Mutual Transport Layer Security(mTLS)

* using a confidential certification authority to ensure the identity of each participating site and to ensure that they meet the computing requirements for confidential computing

* defining additional roles to manage who can submit which kind of jobs to execute within NVFlare, identify who submits jobs and which dataset can be accessed

Users can write their own FLComponents, listening to the NVFlare system events at different points of their workflow, then easily plug in their authentication and authorization logic as needed.

### Assumptions and Risks
By enabling the customized site-specific authentication and authorization, NVFlare will make several security related data available to the external FL components, e.g. IDENTITY_NAME, PUBLIC_KEY, CERTIFICATE, etc. In order to protect them from being compromised, that data needs to be made read-only.

Because of the external pluginable authentication and authorization processes, the results of the processes could potentially cause the jobs to not be able to be deployed or run. When configuring and using these functions, the users need to be aware of the impact and know where to plug in the authentication and authorization check.

### Event based pluginable authentication and authorization
The NVFlare event based solution supports site-specific authentication and federated job-level authorization. Users can provide and implement any sort of additional security checks by building and plugging in FLcomponents which listen to the appropriate events and provide custom authentication and authorization functions.


To use the site-specific security functions, write a custom Security implementation in the local/custom/security_handler.py, then configure it as a component in the site resources.json.

```
from typing import Tuple

from nvflare.apis.event_type import EventType
from nvflare.apis.fl_component import FLComponent
from nvflare.apis.fl_constant import FLContextKey
from nvflare.apis.fl_context import FLContext
from nvflare.apis.job_def import JobMetaKey


class CustomSecurityHandler(FLComponent):

    def handle_event(self, event_type: str, fl_ctx: FLContext):
        if event_type == EventType.AUTHORIZE_COMMAND_CHECK:
            result, reason = self.authorize(fl_ctx=fl_ctx)
            if not result:
                fl_ctx.set_prop(FLContextKey.AUTHORIZATION_RESULT, False, sticky=False)
                fl_ctx.set_prop(FLContextKey.AUTHORIZATION_REASON, reason, sticky=False)

    def authorize(self, fl_ctx: FLContext) -> Tuple[bool, str]:
        command = fl_ctx.get_prop(FLContextKey.COMMAND_NAME)
        if command in ["check_resources"]:
            security_items = fl_ctx.get_prop(FLContextKey.SECURITY_ITEMS)
            job_meta = security_items.get(FLContextKey.JOB_META)
            if job_meta.get(JobMetaKey.JOB_NAME) == "FL Demo Job1":
                return False, f"Not authorized to execute: {command}"
            else:
                return True, ""
        else:
            return True, ""

```

# Site-specific Authentication & Authorization

Lets look these mechanism via examples

## Server Side Customized Authenticaton

In this example, we will see how do we designed a custom plugin with additional authentication check, 
As result for two sites in POC, site-2 is NOT able to start and register to the server. It's blocked by the ServerCustomSecurityHandler logic during the client registration.

### Define a server side security handler

Notice the we the customized the handler raise NotAuthenticated("site_2 not allowed to register")

In [ ]:
!cat code/custom_server_side_authentication/security/server/custom/security_handler.py

To register this plugin handler, we need to add this component to the server site's local configuration

by adding it to the components array 

```
    components: [
    ...
            {
                "id": "security_handler",
                "path": "security_handler.ServerCustomSecurityHandler"
            }
    ]  
```  

In this example, we have made a edit resources.json file and will copy both "custom" folder and "resources.json" to the local directory



In [ ]:
! echo y | nvflare poc prepare

!cp -r code/custom_server_side_authentication/security/server/* /tmp/nvflare/poc/example_project/prod_00/server/local/.

! cat /tmp/nvflare/poc/example_project/prod_00/server/local/resources.json



Now, go to a terminal and try to start FL system with 

```

nvflare poc start -ex admin@nvidia.com
```

See what happens

You should see something like this: 


```
2025-02-02 16:35:40,059 - Communicator - INFO - Trying to register with server ...
2025-02-02 16:35:40,060 - ServerCustomSecurityHandler - ERROR - [identity=server, run=?, peer=site-2, peer_run=?] - Exception when handling event "_client_register_received": NotAuthenticated: site-2 not allowed to register


```

## Client Side: Customized Job-level Auhtorization

Next, let's take look authorization on the client side

**Setup**

* `server`: NVFlare server
* `site-1`: Site-1 has a CustomSecurityHandler set up which does not allow the job "Secret-Job" to run. All other jobs will be able to deploy and run on site-1.
* `site-2`: Site-2 allows any job to be deployed and run.

**Expectation**
* "Secret-Job" will be deployed and run on site-2 but not on site-1



